In [51]:
with open("9f9e37a6-55f4-43c4-8285-b4b976f5dd4b.md", "r", encoding="utf-8") as file:
    content = file.read()

print(content[0:100])


# 든든히 먹고
튼튼히 크는
서울든든급식# SE♡UL M! SOUL# 서울특별시# < 진행 순서 >| 내 용 | 진 행 |
| --- | --- |
| 든든급식 개편사항 안내(10


In [52]:
from dotenv import load_dotenv

load_dotenv()

True

In [53]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

In [54]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap =10,
    separators = ["\n\n", "\n", " ", ""]
)

In [55]:

docs = text_splitter.create_documents([content])

In [56]:
print(len(docs))

23


In [61]:
# Elasticsearch BM25 구현
from elasticsearch import Elasticsearch
import json

# Elasticsearch 클라이언트 연결 (9.x 버전 호환)
es = Elasticsearch(
    hosts=["http://localhost:9200"],
    verify_certs=False,
    ssl_show_warn=False
)

# 연결 확인
try:
    if es.ping():
        print("Elasticsearch에 성공적으로 연결되었습니다!")
        # 클러스터 정보 출력
        info = es.info()
        print(f"Elasticsearch 버전: {info['version']['number']}")
    else:
        print("Elasticsearch 연결에 실패했습니다.")
except Exception as e:
    print(f"Elasticsearch 연결 오류: {e}")
    print("Docker Compose로 Elasticsearch가 실행 중인지 확인해주세요.")


Elasticsearch에 성공적으로 연결되었습니다!
Elasticsearch 버전: 9.0.2


In [62]:
# 인덱스 생성 (BM25 설정 포함)
index_name = "bm25_documents"

# 인덱스 매핑 정의 (BM25 파라미터 설정)
mapping = {
    "settings": {
        "index": {
            "similarity": {
                "bm25_similarity": {
                    "type": "BM25",
                    "k1": 1.2,    # 기본값
                    "b": 0.75     # 기본값
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {
                "type": "text",
                "similarity": "bm25_similarity",
                "analyzer": "standard"
            },
            "chunk_id": {
                "type": "integer"
            }
        }
    }
}

# 인덱스가 존재하면 삭제
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"기존 인덱스 '{index_name}'를 삭제했습니다.")

# 인덱스 생성
es.indices.create(index=index_name, body=mapping)
print(f"인덱스 '{index_name}'를 생성했습니다.")


기존 인덱스 'bm25_documents'를 삭제했습니다.
인덱스 'bm25_documents'를 생성했습니다.


In [63]:
# 문서 chunk들을 Elasticsearch에 인덱싱
for i, doc in enumerate(docs):
    document = {
        "content": doc.page_content,
        "chunk_id": i
    }
    
    # 문서 인덱싱
    response = es.index(index=index_name, id=i, body=document)
    print(f"Chunk {i} 인덱싱 완료")

# 인덱스 새로고침 (검색 가능하도록)
es.indices.refresh(index=index_name)
print(f"총 {len(docs)}개의 chunk가 인덱싱되었습니다.")


Chunk 0 인덱싱 완료
Chunk 1 인덱싱 완료
Chunk 2 인덱싱 완료
Chunk 3 인덱싱 완료
Chunk 4 인덱싱 완료
Chunk 5 인덱싱 완료
Chunk 6 인덱싱 완료
Chunk 7 인덱싱 완료
Chunk 8 인덱싱 완료
Chunk 9 인덱싱 완료
Chunk 10 인덱싱 완료
Chunk 11 인덱싱 완료
Chunk 12 인덱싱 완료
Chunk 13 인덱싱 완료
Chunk 14 인덱싱 완료
Chunk 15 인덱싱 완료
Chunk 16 인덱싱 완료
Chunk 17 인덱싱 완료
Chunk 18 인덱싱 완료
Chunk 19 인덱싱 완료
Chunk 20 인덱싱 완료
Chunk 21 인덱싱 완료
Chunk 22 인덱싱 완료
총 23개의 chunk가 인덱싱되었습니다.


In [64]:
# BM25 검색 함수 정의
def bm25_search(query, top_k=3):
    """
    BM25 알고리즘을 사용하여 문서 검색
    
    Args:
        query (str): 검색 쿼리
        top_k (int): 반환할 상위 결과 개수
    
    Returns:
        list: 검색 결과와 점수
    """
    search_body = {
        "size": top_k,
        "query": {
            "match": {
                "content": {
                    "query": query,
                    "operator": "or"
                }
            }
        },
        "_source": ["content", "chunk_id"],
        "highlight": {
            "fields": {
                "content": {
                    "fragment_size": 150,
                    "number_of_fragments": 3
                }
            }
        }
    }
    
    # 검색 실행
    response = es.search(index=index_name, body=search_body)
    
    results = []
    for hit in response['hits']['hits']:
        result = {
            'chunk_id': hit['_source']['chunk_id'],
            'score': hit['_score'],
            'content': hit['_source']['content'],
            'highlight': hit.get('highlight', {}).get('content', [])
        }
        results.append(result)
    
    return results

# 검색 테스트
test_query = "서울든든급식"
search_results = bm25_search(test_query, top_k=3)

print(f"검색어: '{test_query}'")
print(f"검색 결과 ({len(search_results)}개):")
print("=" * 50)

for i, result in enumerate(search_results, 1):
    print(f"[결과 {i}] Chunk ID: {result['chunk_id']}, BM25 점수: {result['score']:.4f}")
    print(f"내용: {result['content'][:200]}...")
    if result['highlight']:
        print(f"하이라이트: {result['highlight'][0][:150]}...")
    print("-" * 30)


검색어: '서울든든급식'
검색 결과 (3개):
[결과 1] Chunk ID: 7, BM25 점수: 2.8021
내용: 11월 20일(수)부터 '24년 12월분 식재료 주문이 가능합니다.

성동구 어린이집 대상 신규 회원가입 안내 >

회원가입 후 12월 20일(금) 일괄 승인될 예정이며,

승인완료 시에는 [승인완료] 문자가 발송됩니다.

12월 20일(금)부터 '25년 1월분 식재료 주문이 가능합니다.

매월 20일까지 신청 시 익월 1일분부터 주문 가능합니다.

- 서...
하이라이트: <em>서울든든급식</em> 신청 바로가기

# 1-3....
------------------------------
[결과 2] Chunk ID: 0, BM25 점수: 2.6964
내용: # 든든히 먹고
튼튼히 크는
서울든든급식# SE♡UL M! SOUL# 서울특별시# < 진행 순서 >| 내 용 | 진 행 |
| --- | --- |
| 든든급식 개편사항 안내(10분) | 서울시 친환경급식과 공공급식팀장 |
| Q&A(10분) |  |
| 든든급식 웹쇼핑몰 사용법 (30분) | 나라원시스템 |
| Q&A(10분) |  |


' "
서울든든...
하이라이트: # 든든히 먹고
튼튼히 크는
<em>서울든든급식</em># SE♡UL M!...
------------------------------
[결과 3] Chunk ID: 6, BM25 점수: 2.5700
내용: # 4. 식재료 주문 방법# 1 식재료주문 서울든든급식 웹쇼핑몰 이용# · 발주기한 납품일 기준 7일 전까지- x (은평·송파) '24.12월 2일분 주문 시 11월 20일~ 25일까지 주문 가능
- (성동) '25. 1월 2일분 주문 시 12월 20일~26일까지 주문 가능
- · 발주 취소 및 변경 납품일 기준 3일 전까지
- · 클레임 요청 시에는, 배...
하이라이트: 식재료 주문 방법# 1 식재료주문 <em>서울든든급식</em> 웹쇼핑몰 이용# · 발주기한 납품일 기준 7일 전까지- x (은평·송파) '24

In [34]:
# 추가 검색 테스트 - 다양한 쿼리로 테스트
test_queries = [
    "디지털 기술",
    "스타트업",
    "AX브릿지위원회",
    "네트워크 AI"
]

print("다양한 검색어로 BM25 테스트")
print("=" * 60)

for query in test_queries:
    print(f"\n🔍 검색어: '{query}'")
    results = bm25_search(query, top_k=2)
    
    if results:
        for i, result in enumerate(results, 1):
            print(f"  [{i}] Chunk {result['chunk_id']} - 점수: {result['score']:.4f}")
            print(f"      내용: {result['content'][:100]}...")
    else:
        print("  검색 결과가 없습니다.")
    print("-" * 40)


다양한 검색어로 BM25 테스트

🔍 검색어: '디지털 기술'
  [1] Chunk 0 - 점수: 3.1239
      내용: # 「AX브릿지위원회」 소개자료(벤처기업협회 회원소통본부, 2024.10.11.)

데이터, 네트워크, AI 등 첨단 디지털 기술 분야 선도벤처와 유망 스
타트업이 함께 모여 디지...
----------------------------------------

🔍 검색어: '스타트업'
  검색 결과가 없습니다.
----------------------------------------

🔍 검색어: 'AX브릿지위원회'
  [1] Chunk 0 - 점수: 0.7727
      내용: # 「AX브릿지위원회」 소개자료(벤처기업협회 회원소통본부, 2024.10.11.)

데이터, 네트워크, AI 등 첨단 디지털 기술 분야 선도벤처와 유망 스
타트업이 함께 모여 디지...
  [2] Chunk 1 - 점수: 0.6386
      내용: # □ 운영조직- ○ 의 장 : 이주완 대표(메가존클라우드)
- ○ 운영위원 : 10명 내외(추가섭외중)
- ○ 사 무 국 : 벤처기업협회 회원소통본부
# □ 주요활동- 1. (정...
----------------------------------------

🔍 검색어: '네트워크 AI'
  [1] Chunk 0 - 점수: 1.7146
      내용: # 「AX브릿지위원회」 소개자료(벤처기업협회 회원소통본부, 2024.10.11.)

데이터, 네트워크, AI 등 첨단 디지털 기술 분야 선도벤처와 유망 스
타트업이 함께 모여 디지...
  [2] Chunk 1 - 점수: 1.2759
      내용: # □ 운영조직- ○ 의 장 : 이주완 대표(메가존클라우드)
- ○ 운영위원 : 10명 내외(추가섭외중)
- ○ 사 무 국 : 벤처기업협회 회원소통본부
# □ 주요활동- 1. (정...
----------------------------------------


In [38]:
# BM25 파라미터 조정 테스트 (수정된 버전)
def update_bm25_parameters(k1=1.2, b=0.75):
    """
    BM25 파라미터를 조정하는 함수 (인덱스 닫기/열기 방식)
    
    Args:
        k1 (float): term frequency 조정 파라미터 (기본값: 1.2)
        b (float): document length normalization 파라미터 (기본값: 0.75)
    """
    try:
        # 1. 인덱스 닫기
        print(f"인덱스 '{index_name}' 닫는 중...")
        es.indices.close(index=index_name)
        
        # 2. 설정 업데이트
        settings = {
            "index": {
                "similarity": {
                    "bm25_similarity": {
                        "type": "BM25",
                        "k1": k1,
                        "b": b
                    }
                }
            }
        }
        
        es.indices.put_settings(index=index_name, body=settings)
        
        # 3. 인덱스 다시 열기
        print(f"인덱스 '{index_name}' 다시 여는 중...")
        es.indices.open(index=index_name)
        
        # 4. 인덱스 새로고침
        es.indices.refresh(index=index_name)
        
        print(f"✅ BM25 파라미터 업데이트 완료: k1={k1}, b={b}")
        
    except Exception as e:
        print(f"❌ BM25 파라미터 업데이트 실패: {e}")
        # 인덱스가 닫혀있을 수 있으니 다시 열어주기
        try:
            es.indices.open(index=index_name)
        except:
            pass

# 서로 다른 BM25 파라미터로 테스트
query = "벤처기업협회"
print(f"테스트 쿼리: '{query}'")
print("=" * 50)

# 기본 파라미터 (k1=1.2, b=0.75)
print("\n📊 기본 BM25 파라미터 (k1=1.2, b=0.75):")
results = bm25_search(query, top_k=2)
for i, result in enumerate(results, 1):
    print(f"  [{i}] Chunk {result['chunk_id']} - 점수: {result['score']:.4f}")

# 파라미터 조정 1 (k1=2.0, b=0.5)
print("\n📊 조정된 BM25 파라미터 (k1=2.0, b=0.5):")
update_bm25_parameters(k1=2.0, b=0.5)
results = bm25_search(query, top_k=2)
for i, result in enumerate(results, 1):
    print(f"  [{i}] Chunk {result['chunk_id']} - 점수: {result['score']:.4f}")

# 기본값으로 복구
print("\n📊 기본값으로 복구:")
update_bm25_parameters(k1=1.2, b=0.75)


테스트 쿼리: '벤처기업협회'

📊 기본 BM25 파라미터 (k1=1.2, b=0.75):
  [1] Chunk 1 - 점수: 0.4632
  [2] Chunk 0 - 점수: 0.3976

📊 조정된 BM25 파라미터 (k1=2.0, b=0.5):
인덱스 'bm25_documents' 닫는 중...
인덱스 'bm25_documents' 다시 여는 중...
✅ BM25 파라미터 업데이트 완료: k1=2.0, b=0.5
  [1] Chunk 1 - 점수: 0.5085
  [2] Chunk 0 - 점수: 0.3894

📊 기본값으로 복구:
인덱스 'bm25_documents' 닫는 중...
인덱스 'bm25_documents' 다시 여는 중...
✅ BM25 파라미터 업데이트 완료: k1=1.2, b=0.75


In [37]:
# 인덱싱된 문서들 확인
print("📄 인덱싱된 문서 내용 확인:")
print("=" * 50)

for i, doc in enumerate(docs):
    print(f"\n[Chunk {i}]")
    print(f"길이: {len(doc.page_content)} 문자")
    print(f"내용: {doc.page_content[:150]}...")
    print("-" * 30)

# 통계 정보
print(f"\n📊 통계:")
print(f"- 총 문서 수: {len(docs)}")
print(f"- 평균 문서 길이: {sum(len(doc.page_content) for doc in docs) / len(docs):.0f} 문자")
print(f"- 최단 문서: {min(len(doc.page_content) for doc in docs)} 문자")
print(f"- 최장 문서: {max(len(doc.page_content) for doc in docs)} 문자")


📄 인덱싱된 문서 내용 확인:

[Chunk 0]
길이: 470 문자
내용: # 「AX브릿지위원회」 소개자료(벤처기업협회 회원소통본부, 2024.10.11.)

데이터, 네트워크, AI 등 첨단 디지털 기술 분야 선도벤처와 유망 스
타트업이 함께 모여 디지털시대의 경쟁우위 확보를 위해 필요한 전략
적 협력을 추구하고 규제 정비와 정책적 지원을 ...
------------------------------

[Chunk 1]
길이: 997 문자
내용: # □ 운영조직- ○ 의 장 : 이주완 대표(메가존클라우드)
- ○ 운영위원 : 10명 내외(추가섭외중)
- ○ 사 무 국 : 벤처기업협회 회원소통본부
# □ 주요활동- 1. (정기포럼) 국내외 업계동향 및 선도벤처 기업사례 중심의 정보교류 목
- 적의 정기포럼 2회(...
------------------------------

[Chunk 2]
길이: 760 문자
내용: | <운영위원> ○ 아시아 태평양 고성장 기업 3년 연속 선정 ○ 아시아' 30세 이하 리더 30인 선정 (포브스) ○ 직가맹점 200개 ↑ 해외 100호점 돌파 ○ 올해 매장 수 500호점 돌파 예정 ○ 누적 투자 유치 600억원 |
| 4 | (주)뤼튼 테크놀로 지...
------------------------------

[Chunk 3]
길이: 856 문자
내용: | 순서 | 소속/성명 | 사진 | 주요약력 |
| --- | --- | --- | --- |
| 8 | (주)스파이어 테크놀로지 강군화 대표이사 | ![image](/image/placeholder)
 | <운영위원> ○ (주)스파이어테크놀로지, (주)스테이지7 대표...
------------------------------

📊 통계:
- 총 문서 수: 4
- 평균 문서 길이: 771 문자
- 최단 문서: 470 문자
- 최장 문서: 997 문자


In [65]:
# 한글 최적화 BM25 인덱스 생성 (Nori 분석기 사용)
korean_index_name = "bm25_korean_documents"

# Nori 분석기가 포함된 인덱스 매핑 정의
korean_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "korean_bm25_similarity": {
                    "type": "BM25",
                    "k1": 1.2,
                    "b": 0.75
                }
            }
        },
        "analysis": {
            "analyzer": {
                "nori_analyzer": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "filter": [
                        "nori_part_of_speech",
                        "nori_readingform",
                        "lowercase"
                    ]
                }
            },
            "tokenizer": {
                "nori_tokenizer": {
                    "type": "nori_tokenizer",
                    "decompound_mode": "mixed"
                }
            },
            "filter": {
                "nori_part_of_speech": {
                    "type": "nori_part_of_speech",
                    "stoptags": [
                        "E", "IC", "J", "MAG", "MAJ", "MM", 
                        "SP", "SSC", "SSO", "SC", "SE", "XPN", "XSA", "XSN", "XSV",
                        "UNA", "NA", "VSV"
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {
                "type": "text",
                "similarity": "korean_bm25_similarity",
                "analyzer": "nori_analyzer",
                "search_analyzer": "nori_analyzer"
            },
            "chunk_id": {
                "type": "integer"
            }
        }
    }
}

# 기존 한글 인덱스가 존재하면 삭제
if es.indices.exists(index=korean_index_name):
    es.indices.delete(index=korean_index_name)
    print(f"기존 한글 인덱스 '{korean_index_name}'를 삭제했습니다.")

# 한글 최적화 인덱스 생성
try:
    es.indices.create(index=korean_index_name, body=korean_mapping)
    print(f"✅ 한글 최적화 인덱스 '{korean_index_name}'를 생성했습니다.")
except Exception as e:
    print(f"❌ 인덱스 생성 실패: {e}")
    print("Nori 플러그인이 설치되지 않았을 수 있습니다.")


❌ 인덱스 생성 실패: BadRequestError(400, 'illegal_argument_exception', 'No enum constant org.apache.lucene.analysis.ko.POS.Tag.E')
Nori 플러그인이 설치되지 않았을 수 있습니다.


In [ ]:
# N-gram 방식으로 한글 최적화 BM25 구현 (Nori 대안)
ngram_index_name = "bm25_ngram_documents"

# N-gram 기반 인덱스 매핑 정의
ngram_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "ngram_bm25_similarity": {
                    "type": "BM25",
                    "k1": 1.2,
                    "b": 0.75
                }
            }
        },
        "analysis": {
            "analyzer": {
                "korean_ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": [
                        "lowercase",
                        "korean_ngram_filter"
                    ]
                },
                "korean_search_analyzer": {
                    "type": "custom", 
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "korean_ngram_filter"
                    ]
                }
            },
            "filter": {
                "korean_ngram_filter": {
                    "type": "ngram",
                    "min_gram": 2,
                    "max_gram": 3
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {
                "type": "text",
                "similarity": "ngram_bm25_similarity",
                "analyzer": "korean_ngram_analyzer",
                "search_analyzer": "korean_search_analyzer"
            },
            "chunk_id": {
                "type": "integer"
            }
        }
    }
}

# N-gram 인덱스 생성
if es.indices.exists(index=ngram_index_name):
    es.indices.delete(index=ngram_index_name)
    print(f"기존 N-gram 인덱스 '{ngram_index_name}'를 삭제했습니다.")

es.indices.create(index=ngram_index_name, body=ngram_mapping)
print(f"✅ N-gram 인덱스 '{ngram_index_name}'를 생성했습니다.")

# N-gram 인덱스에 문서 인덱싱
print("N-gram 인덱스에 문서 인덱싱 중...")
for i, doc in enumerate(docs):
    document = {
        "content": doc.page_content,
        "chunk_id": i
    }
    es.index(index=ngram_index_name, id=i, body=document)
    print(f"N-gram Chunk {i} 인덱싱 완료")

es.indices.refresh(index=ngram_index_name)
print(f"✅ N-gram 인덱스에 {len(docs)}개의 chunk가 인덱싱되었습니다.")


In [ ]:
# N-gram BM25 검색 함수와 성능 비교
def ngram_bm25_search(query, top_k=3):
    """N-gram BM25 검색"""
    search_body = {
        "size": top_k,
        "query": {
            "match": {
                "content": {
                    "query": query,
                    "operator": "or"
                }
            }
        },
        "_source": ["content", "chunk_id"],
        "highlight": {
            "fields": {
                "content": {
                    "fragment_size": 150,
                    "number_of_fragments": 3
                }
            }
        }
    }
    
    response = es.search(index=ngram_index_name, body=search_body)
    results = []
    for hit in response['hits']['hits']:
        result = {
            'chunk_id': hit['_source']['chunk_id'],
            'score': hit['_score'],
            'content': hit['_source']['content'],
            'highlight': hit.get('highlight', {}).get('content', [])
        }
        results.append(result)
    return results

# Standard vs N-gram 분석기 성능 비교
print("🔍 Standard vs N-gram 분석기 성능 비교")
print("=" * 70)

test_queries = ["서울든든급식", "식재료 주문", "어린이집", "급식", "든든"]

for query in test_queries:
    print(f"\n📝 검색어: '{query}'")
    print("-" * 50)
    
    # Standard 분석기 결과
    print("🔹 Standard 분석기:")
    standard_results = bm25_search(query, top_k=2)
    if standard_results:
        for i, result in enumerate(standard_results, 1):
            print(f"  [{i}] Chunk {result['chunk_id']} - 점수: {result['score']:.4f}")
    else:
        print("  검색 결과 없음")
    
    # N-gram 분석기 결과
    print("🔹 N-gram 분석기:")
    ngram_results = ngram_bm25_search(query, top_k=2)
    if ngram_results:
        for i, result in enumerate(ngram_results, 1):
            print(f"  [{i}] Chunk {result['chunk_id']} - 점수: {result['score']:.4f}")
    else:
        print("  검색 결과 없음")
    
    print("=" * 50)


In [ ]:
# 토크나이저 분석 결과 비교
print("\n🔍 토크나이저 분석 결과 비교")
print("=" * 60)

def analyze_text(text, analyzer_name, index_name):
    """텍스트를 분석하여 토큰 결과를 반환"""
    try:
        response = es.indices.analyze(
            index=index_name,
            body={
                "analyzer": analyzer_name,
                "text": text
            }
        )
        tokens = [token['token'] for token in response['tokens']]
        return tokens
    except Exception as e:
        print(f"분석 실패: {e}")
        return []

# 분석할 텍스트 샘플
sample_texts = [
    "서울든든급식",
    "식재료 주문 방법",
    "어린이집 대상",
    "든든급식 개편사항"
]

for text in sample_texts:
    print(f"\n📝 분석 텍스트: '{text}'")
    print("-" * 40)
    
    # Standard 분석기 결과
    standard_tokens = analyze_text(text, "standard", index_name)
    print(f"🔹 Standard: {standard_tokens}")
    
    # N-gram 분석기 결과
    ngram_tokens = analyze_text(text, "korean_ngram_analyzer", ngram_index_name)
    print(f"🔹 N-gram:   {ngram_tokens[:10]}{'...' if len(ngram_tokens) > 10 else ''}")  # 처음 10개만 표시
    
    # 차이점 분석
    print(f"💡 토큰 개수: Standard({len(standard_tokens)}) vs N-gram({len(ngram_tokens)})")
    
    print("-" * 40)


In [ ]:
# Nori 플러그인 설치 방법 안내
print("\n📋 Nori 플러그인 설치 방법")
print("=" * 60)
print("""
Nori는 Elasticsearch의 공식 한국어 분석 플러그인입니다.
현재 Docker 환경에서 Nori를 설치하려면 다음 방법을 사용하세요:

🛠️ 방법 1: Docker Compose 수정
docker-compose.yaml 파일을 다음과 같이 수정:

```yaml
services:
  elasticsearch:
    image: docker.elastic.co/elasticsearch/elasticsearch:9.0.2
    container_name: elasticsearch-local
    environment:
      - discovery.type=single-node
      - ES_JAVA_OPTS=-Xms1g -Xmx1g
      - xpack.security.enabled=false
      - xpack.security.http.ssl.enabled=false
    ulimits:
      memlock:
        soft: -1
        hard: -1
    volumes:
      - esdata:/usr/share/elasticsearch/data
    ports:
     - "9200:9200"
     - "9300:9300"
    command: >
      /bin/bash -c "
      elasticsearch-plugin install analysis-nori &&
      /usr/local/bin/docker-entrypoint.sh eswrapper
      "
```

🛠️ 방법 2: 실행 중인 컨테이너에 설치
```bash
# 컨테이너에 접속
docker exec -it elasticsearch-local bash

# Nori 플러그인 설치
elasticsearch-plugin install analysis-nori

# Elasticsearch 재시작
exit
docker-compose restart elasticsearch
```

🛠️ 방법 3: 새로운 이미지 빌드
Dockerfile을 만들어 Nori가 포함된 이미지 생성

⚠️ 현재 상황에서는 N-gram 방식으로도 충분히 한글 BM25 성능을 향상시킬 수 있습니다!
""")

print("\n💡 N-gram vs Nori 비교")
print("=" * 40)
print("""
✅ N-gram 방식 (현재 구현):
- 설치 불필요, 즉시 사용 가능
- 부분 매칭에 강함 ("급식" 검색으로 "든든급식" 찾기)
- 2-3글자 단위로 분리하여 검색 범위 확장
- 노이즈가 많을 수 있음

✅ Nori 방식 (권장):
- 형태소 분석으로 의미 단위 분리
- 불용어 제거로 검색 정확도 향상
- 복합어 분해 (예: "든든급식" → "든든" + "급식")
- 더 정교한 한글 검색 지원
""")

print("\n🎯 결론: N-gram으로도 기본 대비 큰 성능 향상을 기대할 수 있습니다!")


In [ ]:
# 한글 최적화 인덱스에 문서 인덱싱
print("한글 최적화 인덱스에 문서 인덱싱 중...")
for i, doc in enumerate(docs):
    document = {
        "content": doc.page_content,
        "chunk_id": i
    }
    
    try:
        # 한글 최적화 인덱스에 문서 인덱싱
        response = es.index(index=korean_index_name, id=i, body=document)
        print(f"한글 최적화 Chunk {i} 인덱싱 완료")
    except Exception as e:
        print(f"Chunk {i} 인덱싱 실패: {e}")
        break

# 한글 최적화 인덱스 새로고침
try:
    es.indices.refresh(index=korean_index_name)
    print(f"✅ 한글 최적화 인덱스에 총 {len(docs)}개의 chunk가 인덱싱되었습니다.")
except Exception as e:
    print(f"❌ 인덱스 새로고침 실패: {e}")


In [ ]:
# 한글 최적화 BM25 검색 함수
def korean_bm25_search(query, top_k=3):
    """
    한글 최적화 BM25 알고리즘을 사용하여 문서 검색
    
    Args:
        query (str): 검색 쿼리
        top_k (int): 반환할 상위 결과 개수
    
    Returns:
        list: 검색 결과와 점수
    """
    search_body = {
        "size": top_k,
        "query": {
            "match": {
                "content": {
                    "query": query,
                    "operator": "or"
                }
            }
        },
        "_source": ["content", "chunk_id"],
        "highlight": {
            "fields": {
                "content": {
                    "fragment_size": 150,
                    "number_of_fragments": 3
                }
            }
        }
    }
    
    try:
        # 한글 최적화 인덱스에서 검색 실행
        response = es.search(index=korean_index_name, body=search_body)
        
        results = []
        for hit in response['hits']['hits']:
            result = {
                'chunk_id': hit['_source']['chunk_id'],
                'score': hit['_score'],
                'content': hit['_source']['content'],
                'highlight': hit.get('highlight', {}).get('content', [])
            }
            results.append(result)
        
        return results
    except Exception as e:
        print(f"검색 중 오류 발생: {e}")
        return []

# Standard vs Nori 분석기 성능 비교
print("🔍 Standard vs Nori 분석기 성능 비교")
print("=" * 70)

test_queries = [
    "서울든든급식",
    "식재료 주문", 
    "어린이집",
    "급식 개편사항"
]

for query in test_queries:
    print(f"\n📝 검색어: '{query}'")
    print("-" * 50)
    
    # Standard 분석기 결과
    print("🔹 Standard 분석기 (기본):")
    standard_results = bm25_search(query, top_k=3)
    if standard_results:
        for i, result in enumerate(standard_results, 1):
            print(f"  [{i}] Chunk {result['chunk_id']} - 점수: {result['score']:.4f}")
    else:
        print("  검색 결과 없음")
    
    # Nori 분석기 결과 (한글 최적화)
    print("🔹 Nori 분석기 (한글 최적화):")
    korean_results = korean_bm25_search(query, top_k=3)
    if korean_results:
        for i, result in enumerate(korean_results, 1):
            print(f"  [{i}] Chunk {result['chunk_id']} - 점수: {result['score']:.4f}")
    else:
        print("  검색 결과 없음")
    
    print("=" * 50)


In [ ]:
# 토크나이저 분석 결과 비교
print("\n🔍 토크나이저 분석 결과 비교")
print("=" * 60)

# 분석할 텍스트 샘플
sample_texts = [
    "서울든든급식",
    "식재료 주문 방법",
    "어린이집 대상 신규 회원가입",
    "든든급식 개편사항 안내"
]

def analyze_text(text, analyzer_name, index_name):
    """텍스트를 분석하여 토큰 결과를 반환"""
    try:
        response = es.indices.analyze(
            index=index_name,
            body={
                "analyzer": analyzer_name,
                "text": text
            }
        )
        tokens = [token['token'] for token in response['tokens']]
        return tokens
    except Exception as e:
        print(f"분석 실패: {e}")
        return []

for text in sample_texts:
    print(f"\n📝 분석 텍스트: '{text}'")
    print("-" * 40)
    
    # Standard 분석기 결과
    standard_tokens = analyze_text(text, "standard", index_name)
    print(f"🔹 Standard: {standard_tokens}")
    
    # Nori 분석기 결과
    korean_tokens = analyze_text(text, "nori_analyzer", korean_index_name)
    print(f"🔹 Nori:     {korean_tokens}")
    
    # 차이점 분석
    if len(korean_tokens) != len(standard_tokens):
        print(f"💡 토큰 개수 차이: Standard({len(standard_tokens)}) vs Nori({len(korean_tokens)})")
    
    print("-" * 40)


In [ ]:
# N-gram 토크나이저를 사용한 대안 방법 (Nori가 없는 경우)
ngram_index_name = "bm25_ngram_documents"

# N-gram 기반 인덱스 매핑 정의
ngram_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "ngram_bm25_similarity": {
                    "type": "BM25",
                    "k1": 1.2,
                    "b": 0.75
                }
            }
        },
        "analysis": {
            "analyzer": {
                "korean_ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": [
                        "lowercase",
                        "korean_ngram_filter"
                    ]
                }
            },
            "filter": {
                "korean_ngram_filter": {
                    "type": "ngram",
                    "min_gram": 2,
                    "max_gram": 3
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {
                "type": "text",
                "similarity": "ngram_bm25_similarity",
                "analyzer": "korean_ngram_analyzer",
                "search_analyzer": "korean_ngram_analyzer"
            },
            "chunk_id": {
                "type": "integer"
            }
        }
    }
}

# N-gram 인덱스 생성
if es.indices.exists(index=ngram_index_name):
    es.indices.delete(index=ngram_index_name)
    print(f"기존 N-gram 인덱스 '{ngram_index_name}'를 삭제했습니다.")

es.indices.create(index=ngram_index_name, body=ngram_mapping)
print(f"✅ N-gram 인덱스 '{ngram_index_name}'를 생성했습니다.")

# N-gram 인덱스에 문서 인덱싱
for i, doc in enumerate(docs):
    document = {
        "content": doc.page_content,
        "chunk_id": i
    }
    es.index(index=ngram_index_name, id=i, body=document)

es.indices.refresh(index=ngram_index_name)
print(f"✅ N-gram 인덱스에 {len(docs)}개의 chunk가 인덱싱되었습니다.")


In [ ]:
# 3가지 토크나이저 성능 종합 비교
def ngram_bm25_search(query, top_k=3):
    """N-gram BM25 검색"""
    search_body = {
        "size": top_k,
        "query": {
            "match": {
                "content": {
                    "query": query,
                    "operator": "or"
                }
            }
        },
        "_source": ["content", "chunk_id"]
    }
    
    response = es.search(index=ngram_index_name, body=search_body)
    results = []
    for hit in response['hits']['hits']:
        result = {
            'chunk_id': hit['_source']['chunk_id'],
            'score': hit['_score'],
            'content': hit['_source']['content']
        }
        results.append(result)
    return results

print("\n🏆 3가지 토크나이저 성능 종합 비교")
print("=" * 80)

comparison_queries = [
    "서울든든급식",
    "급식 개편",
    "식재료 주문",
    "어린이집 회원가입"
]

for query in comparison_queries:
    print(f"\n🔍 검색어: '{query}'")
    print("=" * 60)
    
    # 1. Standard 분석기
    print("1️⃣ Standard 분석기 (기본):")
    standard_results = bm25_search(query, top_k=2)
    if standard_results:
        for i, result in enumerate(standard_results, 1):
            print(f"   [{i}] Chunk {result['chunk_id']} - 점수: {result['score']:.4f}")
            print(f"       내용: {result['content'][:80]}...")
    else:
        print("   검색 결과 없음")
    
    # 2. Nori 분석기 (한글 형태소 분석)
    print("\\n2️⃣ Nori 분석기 (한글 형태소 분석):")
    korean_results = korean_bm25_search(query, top_k=2)
    if korean_results:
        for i, result in enumerate(korean_results, 1):
            print(f"   [{i}] Chunk {result['chunk_id']} - 점수: {result['score']:.4f}")
            print(f"       내용: {result['content'][:80]}...")
    else:
        print("   검색 결과 없음 (Nori 플러그인 미설치)")
    
    # 3. N-gram 분석기
    print("\\n3️⃣ N-gram 분석기 (2-3글자 단위):")
    ngram_results = ngram_bm25_search(query, top_k=2)
    if ngram_results:
        for i, result in enumerate(ngram_results, 1):
            print(f"   [{i}] Chunk {result['chunk_id']} - 점수: {result['score']:.4f}")
            print(f"       내용: {result['content'][:80]}...")
    else:
        print("   검색 결과 없음")
    
    print("\\n" + "=" * 60)
